In [1]:
from pycaret.utils import version
version()

2.0


In [2]:
from pycaret.datasets import get_data
data = get_data('boston')

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
from pycaret.anomaly import *
ano1 = setup(data, normalize = True, silent=True, html=False, session_id=123)

Setup Succesfully Completed!
                     Description      Value
0                    session_id         123
1                 Original Data   (506, 14)
2                Missing Values       False
3              Numeric Features          12
4          Categorical Features           2
5              Ordinal Features       False
6     High Cardinality Features       False
7              Transformed Data   (506, 23)
8               Numeric Imputer        mean
9           Categorical Imputer    constant
10                    Normalize        True
11             Normalize Method      zscore
12               Transformation       False
13        Transformation Method        None
14                          PCA       False
15                   PCA Method        None
16               PCA components        None
17          Ignore Low Variance       False
18          Combine Rare Levels       False
19         Rare Level Threshold        None
20              Numeric Binning       False
21 

In [4]:
create_model('iforest')

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=1,
    random_state=123, verbose=0)

In [5]:
create_model('knn')

KNN(algorithm='auto', contamination=0.05, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [12]:
t = tune_model('iforest', supervised_target = 'medv', custom_grid = [0.02,0.04,0.07], estimator = 'xgboost')

IntProgress(value=0, description='Processing: ', max=18)

Output()

Best Model: Isolation Forest | Fraction %: 0.04 | R2 : 0.7481


In [13]:
get_config('X')

,crim,zn,indus,nox,rm,age,dis,tax,ptratio,black,...,rad_1,rad_2,rad_24,rad_3,rad_4,rad_5,rad_6,rad_7,rad_8,Score
0,-0.419782,0.284830,-1.287909,-0.144217,0.413672,-0.120013,0.140214,-0.666608,-1.459000,0.441052,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.920219
1,-0.417339,-0.487722,-0.593381,-0.740262,0.194274,0.367166,0.557160,-0.987329,-0.303094,0.441052,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.486806
2,-0.417342,-0.487722,-0.593381,-0.740262,1.282714,-0.265812,0.557160,-0.987329,-0.303094,0.396427,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.355687
3,-0.416750,-0.487722,-1.306878,-0.835284,1.016303,-0.809889,1.077737,-1.106115,0.113032,0.416163,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.278615
4,-0.412482,-0.487722,-1.306878,-0.835284,1.228577,-0.511180,1.077737,-1.106115,0.113032,0.441052,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.334761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.413229,-0.487722,0.115738,0.158124,0.439316,0.018673,-0.625796,-0.803212,1.176466,0.387217,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.636636
502,-0.415249,-0.487722,0.115738,0.158124,-0.234548,0.288933,-0.716639,-0.803212,1.176466,0.441052,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.639357
503,-0.413447,-0.487722,0.115738,0.158124,0.984960,0.797449,-0.773684,-0.803212,1.176466,0.441052,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840075
504,-0.407764,-0.487722,0.115738,0.158124,0.725672,0.736996,-0.668437,-0.803212,1.176466,0.403225,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.782062


In [14]:
get_config('seed')

123

In [ ]:
tune_model('kmeans', supervised_target = 'Age')

In [ ]:
save_model(kmeans, 'kmeans')

In [ ]:
from pycaret.clustering import load_model
model = load_model('kmeans')

In [ ]:
from pycaret.clustering import set_config
set_config('X', data)
set_config('data_', data)
set_config('seed', 123)

In [ ]:
from pycaret.clustering import plot_model
plot_model(model[1], plot = 'distribution')

In [ ]:
import logging
logger = logging.getLogger('logs')

In [ ]:
try:
    k = logger.name

In [ ]:
try:
    hasattr(logger, 'name')
except:
    print('hello')

In [ ]:
lda = create_model('lda')

In [ ]:
import inspect
d = inspect.getmembers(lda)

In [ ]:
dic = d[2]

In [ ]:
dic = dic[1]

In [ ]:
dic

In [ ]:
allowed_models = ['kmeans', 'sc', 'hclust', 'birch', 'kmodes']

In [ ]:
for i in allowed_models:
    tune_model(model = i, supervised_target = 'Price', estimator = 'lightgbm')

In [ ]:
ap = create_model('ap')

In [ ]:
plot_model(ap, plot='distribution')

In [ ]:
plot_model(ap)

In [ ]:
optics = create_model('optics')

In [ ]:
plot_model(optics, plot = 'distribution')

In [ ]:
plot_model(optics)

In [ ]:
kmodes = create_model('kmodes')

In [ ]:
kmeans = create_model('kmeans', ground_truth='medv')

In [ ]:
kmeans = create_model('kmeans', num_clusters=2, ground_truth='Purchase')

In [ ]:
n = [2,3,4,5,6,7,8,9,10]
for i in n:
    create_model('kmeans', num_clusters=i, ground_truth='Purchase')

In [ ]:
n = [2,3,4,5,6,7,8,9,10]
for i in n:
    create_model('kmeans', num_clusters=i)

In [ ]:
kmeans, kmeans_result = create_model('kmeans')

In [ ]:
d = kmeans_result.to_dict().get('Metric')

In [ ]:
t = tune_model(model='kmeans', supervised_target='medv', estimator='lightgbm')

In [ ]:
plot_model(kmodes)

In [ ]:
plot_model(kmodes, plot = 'distribution')

In [ ]:
kmeans = create_model('kmeans', ground_truth='Age')

In [ ]:
plot_model(kmodes, plot = 'tsne')

In [ ]:
a = 1
b = 2
c = 3

In [ ]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
pd.DataFrame({'a' : 1, 'b' : 2}) 

In [ ]:
df

In [ ]:
ld = ['Silhouette Score', 'Calinski Harabasz', 'Davies-Bouldin', 'Homogeneity Score']
ldv = [1,2,3,1]

In [ ]:
df = pd.DataFrame(ldv)
df.columns = ['Metric']

In [ ]:
df.set_index([ld])

In [ ]:
df.to_dict()

In [ ]:
df

In [ ]:
kmodes = create_model('dbscan')

In [ ]:
kmodes.labels_

In [ ]:
k = False

In [ ]:
l = k

In [ ]:
l

In [ ]:
plot_model(kmeans, save=True)

In [ ]:
plot_model(kmeans, plot='tsne', save=True)

In [ ]:
plot_model(kmeans, plot='distance', save=True)

In [ ]:
plot_model(kmeans, plot='elbow', save=True)

In [ ]:
plot_model(kmeans, plot='distribution', save=True)

In [ ]:
plot_model(kmeans, plot='silhouette', save=True)

In [ ]:
s[8]

In [ ]:
kmodes = create_model('kmodes')

In [ ]:
meanshift = create_model('birch')

In [ ]:
meanshift = create_model('birch')

In [ ]:
plot_model(kmeans)

In [ ]:
lr = create_model('lr')

In [ ]:
call = compare_models(n_select=10)

In [ ]:
automl()

In [ ]:
automl(use_holdout=True)

In [ ]:
predict_model(create_model('lr', verbose=False));

In [ ]:
level1 = call[:3]
level2 = call[3:6]
level3 = call[6:]

In [ ]:
blend1 = blend_models(level1)

In [ ]:
blend2 = blend_models(level2)

In [ ]:
blend3 = blend_models(level3)

In [ ]:
blend_all = blend_models()

In [ ]:
automl()

In [ ]:
automl(use_holdout=True)

In [ ]:
stack1 = stack_models(estimator_list=call[:3], meta_model=call[0])

In [ ]:
pull()

In [ ]:
predict_model(stack1)

In [ ]:
pull()

In [ ]:
stack1

In [ ]:
stack2 = stack_models(estimator_list=level2, meta_model=level1[0])

In [ ]:
stack3 = stack_models(estimator_list=level3, meta_model=level1[0])

In [ ]:
stack4 = stack_models(estimator_list=level2, meta_model=level1[0], choose_better=False)

In [ ]:
pull()

In [ ]:
automl()

In [ ]:
automl(use_holdout=True)

In [ ]:
stacknet = create_stacknet(estimator_list=[level1,level2], meta_model=level1[0])

In [ ]:
pull()

In [ ]:
automl()

In [ ]:
automl(use_holdout=True)